In [1]:
import pyscreenshot as ImageGrab
import time
from PIL import Image
import cv2
import pandas as pd
import re
import numpy as np
import math
from pathlib import Path  
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [2]:
def line_clear_type(line_diff):
    if line_diff== 1:
        return 'Single'
    elif line_diff==2:
        return 'Double'
    elif line_diff==3:
        return "Triple"
    elif line_diff==4:
        return 'Tetris'
    else:
        return np.nan

In [8]:
d = {'score':['0'], 'lines':[0]}
df=pd.DataFrame(data=d)

time.sleep(5)

try:
    while True:
        
        #take screen shot of # lines
        #im2=ImageGrab.grab(bbox=(900,1113,1000,1145))
        im2=np.array(ImageGrab.grab(bbox=(900,1113,1000,1145)))
        im2 = cv2.cvtColor((im2), cv2.COLOR_BGR2GRAY)
        im2 = cv2.GaussianBlur(im2, (3, 3), 0)
        
        try:
            lines = pd.to_numeric(pytesseract.image_to_string(im2, lang='eng', config='--psm 7 --oem 3 -c tessedit_char_whitelist=0123456789'))
            if math.isnan(lines) == True:
                lines=36
        except:
            pass
    
       # print (lines)
    
        if lines == df['lines'].iloc[-1]:
            pass
        else:
            im1=ImageGrab.grab(bbox=(877,962,1027,995))
            score = int(pytesseract.image_to_string(im1, lang='eng', config='--psm 7 --oem 3 -c tessedit_char_whitelist=0123456789'))
            df2 = pd.DataFrame([[score,lines]], columns=['score','lines'])
            df=pd.concat([df,df2])
    
        time.sleep(0.1)
except KeyboardInterrupt:
    pass

ValueError: invalid literal for int() with base 10: ''

In [9]:
#Make Score numeric (from string)
df['score']=pd.to_numeric(df['score'])
df['lines']=pd.to_numeric(df['lines'])

# Calculate Level you are on (+0.1 makes sure we never have number divisible by 10)
df['level'] = ((df['lines']+0.1)/10).apply(np.ceil)
#df['level'] = df['level'].astype(int)

#Calculate difference in score and lines cleared (r2-r1)
df["score_diff"] = df["score"].diff(1)

df['lines_diff'] = df['lines'].diff(1)

#apply name to line clear type (single,double,triple,tetris)
df['line_clear_type']= df['lines_diff'].apply(line_clear_type)
df['line_clear_type'].iloc[0] = np.nan #set first row to NaN

#use line_clear_type to find if back to back tetris is applied (x0.5 extra points)
df['line_clear_type_s']=df['line_clear_type'].shift(-1)
df['back_to_back'] = (df['line_clear_type'] == df['line_clear_type_s']) & (df['line_clear_type_s']=='Tetris')
df['back_to_back'] = df['back_to_back'].shift(1)



df=df.drop(['line_clear_type_s'], axis=1)
df =df[df['score_diff'] !=0]
df

C:\Users\Reed\AppData\Local\Temp\ipykernel_16088\661551403.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['line_clear_type'].iloc[0] = np.nan #set first row to NaN


,score,lines,level,score_diff,lines_diff,line_clear_type,back_to_back
0,0,0,1.0,NaN,NaN,NaN,NaN
0,1360,4,1.0,1360.0,4.0,Tetris,False
0,2784,8,1.0,1424.0,4.0,Tetris,True
0,4314,12,2.0,1530.0,4.0,Tetris,True
0,6844,16,2.0,2530.0,4.0,Tetris,True
...,...,...,...,...,...,...,...
0,320741,180,19.0,1801.0,1.0,Single,False
0,322641,181,19.0,1900.0,1.0,Single,False
0,324541,182,19.0,1900.0,1.0,Single,False
0,326441,183,19.0,1900.0,1.0,Single,False


In [27]:
im2.show()

In [28]:

im1.show()


In [7]:
time.sleep(3)
im2=np.array(ImageGrab.grab(bbox=(900,1113,1000,1145)))
#im2 = cv2.resize((im2), (50, 50), interpolation = cv2.INTER_LINEAR)
im2 = cv2.cvtColor((im2), cv2.COLOR_BGR2GRAY)
im2 = cv2.GaussianBlur(im2, (3, 3), 0)

cv2.imshow("crp", im2)
cv2.waitKey()

lines = int(pytesseract.image_to_string(im2, lang='eng', config='--psm 7 --oem 3 -c tessedit_char_whitelist=0123456789'))
lines

2

numpy.ndarray

In [10]:
filepath = Path('Tetris_Games/game6.csv')  

df.to_csv(filepath, index=False)